In [1]:
!pip install --upgrade networkx[default]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.2 MB/s eta 0:00:0000:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3


In [2]:
import networkx as nx
import multiprocessing as mp

def find_cliques_in_subgraph(subgraph):
    """
    Find cliques in a subgraph.
    
    Parameters:
    - subgraph: A NetworkX subgraph to find cliques in.
    
    Returns:
    - cliques: A list of cliques found in the subgraph.
    """
    return list(nx.find_cliques(subgraph))

def divide_graph_into_subgraphs(graph, num_partitions):
    """
    Divide a graph into smaller subgraphs for parallel processing.
    
    Parameters:
    - graph: A NetworkX graph.
    - num_partitions: Number of subgraphs to create.
    
    Returns:
    - subgraphs: List of subgraphs to process.
    """
    # If the graph is connected, divide by connected components
    if nx.is_connected(graph):
        return [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    else:
        # For unconnected graphs, we use node-based partitioning
        nodes = list(graph.nodes)
        node_batches = [nodes[i::num_partitions] for i in range(num_partitions)]
        subgraphs = [graph.subgraph(batch).copy() for batch in node_batches]
        return subgraphs

def parallel_clique_detection(graph, num_partitions):
    """
    Parallelize the clique detection using multiprocessing.
    
    Parameters:
    - graph: A NetworkX graph.
    - num_partitions: Number of partitions (processes) for parallel processing.
    
    Returns:
    - all_cliques: List of all cliques found across all subgraphs.
    """
    # Step 1: Divide the graph into subgraphs
    subgraphs = divide_graph_into_subgraphs(graph, num_partitions)

    # Step 2: Set up a pool of workers and apply the function to each subgraph
    with mp.Pool(processes=num_partitions) as pool:
        results = pool.map(find_cliques_in_subgraph, subgraphs)

    # Step 3: Combine the results from each process
    all_cliques = [clique for sublist in results for clique in sublist]
    
    return all_cliques

# Example usage
if __name__ == "__main__":
    # Create or load your graph
    G = nx.barabasi_albert_graph(1000, 5)  # Example graph (use your large graph here)
    
    # Number of processes to use for parallelization
    num_partitions = mp.cpu_count()  # Use all available CPU cores
    
    # Run parallel clique detection
    cliques = parallel_clique_detection(G, num_partitions)
    
    # Output the results
    print(f"Found {len(cliques)} cliques.")


Found 4180 cliques.


In [3]:
num_partitions = mp.cpu_count()  # Use all available CPU cores

In [4]:
num_partitions

72